<a href="https://colab.research.google.com/github/doraemonidol/Colab-Project/blob/main/FCFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import Library

In [185]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# 2. Job Class

In [186]:
class JobSequence:
    def __init__(self, job, processingTime, releaseTime):
        self.job = job
        self.processingTime = processingTime
        self.releaseTime = releaseTime
        self.startTime = 0
        self.duration = 0

# 3. Input

## 3.1. Input from File

In [187]:
jobs = []

with open('data.txt', 'r') as f:
    for line in f:
        job, processingTime, releaseTime = line.split(';')
        jobs.append(JobSequence(job, int(releaseTime), int(processingTime)))

## 3.2. Input from Keyboard

In [ ]:
print("Enter the number of jobs: ")
n = int(input())
jobs = []
for i in range(n):
    print("Enter the processing time and due date for job ", i+1, ":")
    print("  + Job name: ", end = "")
    job = input()
    print("  + Processing Time: ", end = "")
    processingTime = int(input())
    print("  + Release Time: ", end = "")
    releaseTime = int(input())
    jobs.append(JobSequence(job, releaseTime, processingTime))

# 4. Sequencing Jobs

In [172]:
def printJobs(jobs):
    for job in jobs:
        if job != jobs[-1]:
            print(job.job, end = "-")
        else:
            print(job.job, end = "")

In [183]:
def ganntChart(jobs, initialJobs):
    # jobs = jobs.copy()
    # jobs.reverse()

    fig, ax = plt.subplots()
    ax.set_title('Gantt Chart')
    ax.set_xlabel('Time')
    ax.set_ylabel('Job Sequence')
    ax.set_yticks(range(1))
    ax.set_yticklabels([""])
    ax.grid(True)

    cmap = plt.colormaps.get_cmap('Paired')
    colors = [cmap(i / len(jobs)) for i in range(len(initialJobs))]

    totalFlowTime = 0

    for i in range(len(jobs)):
        ax.broken_barh([(totalFlowTime, jobs[i][1])], (i - 0.4, 0.8), facecolors=(colors[initialJobs.index(jobs[i][0])]))
        ax.text(totalFlowTime + jobs[i][1] / 2, i, jobs[i][0], ha='center', va='center', color='black')
        totalFlowTime += jobs[i][1]

    plt.show()

## 4.6. Shortest Remaining Processing Time (SRPT)

In [174]:
srpt_jobs = jobs.copy()
srpt_jobs.sort(key = lambda x: x.releaseTime)

In [ ]:
releaseList = [job.releaseTime for job in srpt_jobs]
releaseTime = releaseList[1:]

current_time = srpt_jobs[0].releaseTime
nextReleaseTime = srpt_jobs[1].releaseTime
scheduled_jobs = []

while len(srpt_jobs) > 0:
    available_jobs = [job for job in srpt_jobs if job.releaseTime <= current_time]
    available_jobs.sort(key = lambda x: x.processingTime)
    # nextTime = min(nextReleaseTime, current_time + available_jobs[0].processingTime)
    nextTime = 0
    if nextReleaseTime == 0:
        nextTime = current_time + available_jobs[0].processingTime
    elif (nextReleaseTime < current_time + available_jobs[0].processingTime):
        nextTime = nextReleaseTime
        nextReleaseTime = 0
        for job in srpt_jobs:
            if job.releaseTime > nextTime:
                nextReleaseTime = job.releaseTime
                break
    else:
        nextTime = current_time + available_jobs[0].processingTime
        nextReleaseTime = 0
        for job in srpt_jobs:
            if job.releaseTime > nextTime:
                nextReleaseTime = job.releaseTime
                break
    available_jobs[0].processingTime -= nextTime - current_time
    scheduled_jobs.append((available_jobs[0].job, nextTime - current_time))
    
    if available_jobs[0].processingTime == 0:
        srpt_jobs.remove(available_jobs[0])
    current_time = nextTime

print("SRPT Schedule: ")
for job in scheduled_jobs:
    print(job[0], ":", job[1], "days")

In [ ]:
print("Objective function value: min(Cmax) = ", current_time, "days")

In [ ]:
ganntChart(scheduled_jobs, [job.job for job in jobs])